In [ ]:
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai.chat_models.azure import AzureChatOpenAI
import os

openai_api_key = os.getenv('AZURE_OPENAI_API_KEY')
openai_api_base = os.getenv('AZURE_OPENAI_ENDPOINT')
openai_api_version = os.getenv('AZURE_OPENAI_API_VERSION')
embeddings_model = os.getenv('AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME')
gpt_model = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

index_path = "../api/index/faiss_index"

embeddings = AzureOpenAIEmbeddings(azure_endpoint=openai_api_base,
                api_key=openai_api_key,
                azure_deployment=embeddings_model)

llm = AzureChatOpenAI(
                openai_api_version=openai_api_version,
                azure_deployment=gpt_model,
            )

@tool
def rag(query:str, k:int = 3):
        """Use this to execute RAG. If the question is related to gen ai in art or music, using this tool retrieve the results."""
        print("Calling RAG...")
        vector_store = FAISS.load_local(index_path, embeddings, allow_dangerous_deserialization=True)
        
        retriever = vector_store.as_retriever()
    
        template = """Answer questions based only on the following context: 
        
        ### CONTEXT ###
        {context}
        
        Question: {question}
        
        If the context doesn't provide enough information to answer the question, say that you don't know.
        Do not respond with information outside the previous context."""

        prompt = PromptTemplate.from_template(template)

        chain = (
            {"context": retriever, "question": RunnablePassthrough()}
            | prompt
            | llm
            | StrOutputParser()
        )

        result = chain.invoke(query)
        return result

In [26]:
@tool
def generate_descriptive_text(input: str):
    """Analyzes input text and re write it as a description to generate diagrams"""
    
    prompt = f"""Your job is to analyze this text: {input} 
    and create a detailed prompt to generate an image of a diagram.
    """
    result = llm.invoke(prompt)
    return result

In [28]:
context = "'AI can generate art by utilizing algorithms and autonomous systems. These systems create artwork either entirely or partially through computational processes. In the case of generative art, artists often write code that defines rules and processes, which can result in unique artworks every time the script is executed. Additionally, AI can use prompt technology to generate content based on user inputs. For example, some AI systems like Midjourney can produce images based on prompts, incorporating various adjustments and iterations to enhance the final piece. Thus, AI serves as a tool that complements the creative process, allowing for the creation of high-quality artistic media across various forms, including digital graphics, music, and literature.'"


generate_descriptive_text(context)

AIMessage(content='**Prompt for Generating an Image of a Diagram:**\n\n---\n\n**Title:** Diagram of AI-Driven Art Generation Processes\n\n**Description:** Create an informative and visually engaging diagram that illustrates the process of how AI generates art. The diagram should include the following elements:\n\n1. **Main Components:**\n   - **AI Algorithms**: Represent this as a box that connects to other elements, indicating that these algorithms form the foundation for generating art.\n   - **Autonomous Systems**: Include a section that describes the role of autonomous systems in the art generation process.\n\n2. **Generative Art Process:**\n   - Visualize a flowchart showing the sequence:\n     - **Code Creation**: An artist writing code.\n     - **Rules and Processes**: A branch leading to a description of how these rules lead to unique outputs.\n     - **Execution of Script**: Indicate that running the code generates artwork.\n\n3. **Prompt Technology:**\n   - Illustrate the use

In [18]:
from openai import AzureOpenAI
from datetime import datetime
import requests
import json
import os
from uuid import uuid4

model = os.getenv('GPT_IMAGES_DEPLOYMENT_NAME')
images_folder = os.getcwd() + "\\assets\\generated_images"

client = AzureOpenAI(azure_endpoint=openai_api_base, 
                     api_key=openai_api_key, 
                     api_version=openai_api_version)

@tool
def generate_images(prompt: str):
        """Generate images, diagrams, charts, etc., based on prompts"""

        sub_folder = (datetime.today().strftime('%Y-%m-%d %H:%M:%S')).replace(" ","-").replace(":","-")
        images_path = os.path.join(images_folder, sub_folder)

        if not os.path.exists(images_path):
            os.makedirs(images_path) 

        result = client.images.generate(
                    model = model,
                    prompt=prompt,
                    n=1
                    )
        
        image_url = json.loads(result.model_dump_json())['data'][0]['url']
        
        image = requests.get(image_url).content
        image_name = f"genai_img_{uuid4()}"

        with open(os.path.join(images_folder, images_path, f"{image_name}.jpg"), 'wb') as handler:
            handler.write(image)
        
        return image_url

In [32]:

rag_agent = create_react_agent(
        model = llm,
        tools = [rag],
        name = "rag_agent"
    )

descriptive_agent = create_react_agent(
        model = llm,
        tools = [generate_descriptive_text],
        name = "descriptive_agent"
    )

image_agent = create_react_agent(
        model = llm,
        tools = [generate_images],
        name = "images_agent"
    )

workflow = create_supervisor(
        agents = [rag_agent, descriptive_agent, image_agent],
        model = llm,
        prompt= ("You are a smart assistant. Your have access to multiple agents," 
                 "your job is to act like a router and decide which agent comes to play."
                 "Here is the list of agents: rag_agent, descriptive_agent, images_agent"
                 "For queries related to gen ai in art, gen ai in music, call the rag_agent"
                 "If queries are unrelated to the previous topics, say that you don't know."
                 "Always handsoff to the descriptive_agent to generate a description for an image based on rag_agent output"
                 "For image, diagrams, charts, first use rag_agent, then descriptive_agent and finally images_agent"
                 "Include the last agent response in yours."  
     )
    )

app =  workflow.compile()

In [ ]:
inputs = {"messages": ["tell me how to create art using generative ai and python"]}
for output in app.stream(inputs):

    for key, value in output.items():
        print(f"output from node {key} ")
        print("----")
        print(value)
        

output from node supervisor 
----
{'messages': [HumanMessage(content='show me a diagram of The research Proces Adapted from Nnizharadze', additional_kwargs={}, response_metadata={}, id='effeb76f-e802-4477-a892-6d3e068db495'), AIMessage(content="I don't know.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 135, 'total_tokens': 141, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_ded0d14823', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'se

In [33]:

query = "give me a list of steps to create art using gen ai and python and return a simple diagram"


response = app.invoke({"messages":[
    {"role": "user", "content": query}
]})


print(response['messages'][-1].content)

Calling RAG...
Here are the steps to create art using generative AI and Python, along with a diagram:

### Steps to Create Art Using Generative AI and Python

1. **Install Python and Required Libraries**:
   - Ensure that Python is installed.
   - Install necessary libraries (e.g., `matplotlib`, `numpy`) using pip.

2. **Define the Generative Art Piece**:
   - Determine the concept/design you want to create.

3. **Import Required Libraries**:
   - In your Python script, import libraries such as `matplotlib` and `numpy`.

4. **Create a Blank Canvas**:
   - Use `NumPy` to create a blank canvas (array) specifying the desired width and height.

5. **Generate the Art**:
   - Define parameters for your art piece and iterate through a range of points.
   - Use polar coordinates or other mathematical functions to manipulate the canvas.
   - Assign random colors to points on the canvas.

6. **Display the Generated Art**:
   - Use `matplotlib` to visualize the generated art and turn off axes for